In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd


'c:\\Chandu\\WorkSpace\\Learnings\\Repos\\End-to-End-CreditCardFraudDetection-Project'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class Model_Config:
    root_dir: Path 
    data_path: Path 
    n_estimators: int
    max_depth: int
    

In [7]:
from creditcard_fraud_detection.constants import *
from creditcard_fraud_detection.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager():
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_model_config(self)-> Model_Config:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])
        model_config = Model_Config(
            root_dir=config.root_dir,
            data_path=config.data_path,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth
        )
        return model_config

In [22]:
import os
import pandas as pd
import numpy as np
from creditcard_fraud_detection.logging import logger
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.metrics import classification_report

In [ ]:
class ModelTrainer():
    def __init__(self,config: Model_Config):
        self.config = config
    
    def train(self):
        model = RandomForestClassifier(n_estimators=self.config.n_estimators, 
                                       max_depth=self.config.max_depth)
        train_df = pd.read_csv(os.path.join(self.config.data_path,'train.csv'))
        train_df_np = train_df.to_numpy()
        X_train = train_df_np[:,:-1]
        y_train = train_df_np[:,-1]
        model.fit(X_train, y_train)
        joblib.dump(model,os.path.join(self.config.root_dir, 'model.pkl'))
        logger.info("model result on the traing data is below:")
        logger.info(classification_report(y_train, model.predict(X_train), target_names=["Not fraud", "Fraud"]))


SyntaxError: f-string: unmatched '[' (1815819572.py, line 14)

In [27]:
try:
    config = ConfigurationManager()
    model_config = config.get_model_config()
    model_trainer = ModelTrainer(config=model_config)
    model_trainer.train()
except Exception as e:
    raise e


[2025-03-27 07:55:01,504: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2025-03-27 07:55:01,518: INFO: common: yaml file: params.yaml loaded succesfully]
[2025-03-27 07:55:01,529: INFO: common: created directory at : artifacts]
[2025-03-27 07:55:01,532: INFO: common: created directory at : artifacts/model_trainer]


[2025-03-27 07:55:02,587: INFO: 2984474853:               precision    recall  f1-score   support

   Not fraud       0.92      1.00      0.96       492
       Fraud       1.00      0.91      0.96       492

    accuracy                           0.96       984
   macro avg       0.96      0.96      0.96       984
weighted avg       0.96      0.96      0.96       984
]
